<a href="https://colab.research.google.com/github/yashkens/ML_Intro_homework/blob/main/HW7_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from sklearn.decomposition import TruncatedSVD
import numpy as np

### Данные

In [62]:
df = pd.read_csv('winemag-data-130k-v2.csv')
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Данных очень много, наверное, для "игрушечного" задания это не так важно, поэтому обрезаю ради экономии времени

In [63]:
print(f'Dataset length: {len(df)}')
df = df.sample(n=10000, random_state=42)
print(f'Dataset Sample length: {len(df)}')

Dataset length: 129971
Dataset Sample length: 10000


В задании не написано, какие иеменно кластеры ожидается получить. Кроме того, в данных нет отзывов, как сказано в задании, поэтому на два класса (negative or positive) тоже поделить получится вряд ли.  
Можно попробовать делить на хорошие-плохие вина, как бы предсказывя критерий points вина. Всего в датасете встречается 21 различное значение points. Я буду пробовать кластеризовать на 21 и на 2 кластера.

In [6]:
points_count = Counter(df['points']).most_common()
points_count

[(87, 1322),
 (88, 1294),
 (90, 1150),
 (89, 961),
 (86, 943),
 (91, 863),
 (92, 747),
 (85, 711),
 (93, 518),
 (84, 480),
 (94, 312),
 (83, 256),
 (82, 164),
 (95, 122),
 (96, 49),
 (81, 43),
 (80, 35),
 (97, 19),
 (98, 7),
 (99, 3),
 (100, 1)]

In [7]:
len(points_count)

21

Векторизую тексты с помощью tf-idf. Затем уменьшаю размерность до 30 составляющих.  
Наверное, если уменьшить ее до 2 составляющих, то мы потеряем очень много информации. А визуализировать 30 размерностей неудобно, поэтому я без визуализаций :( 

In [8]:
vectorizer = TfidfVectorizer()
vec_texts = vectorizer.fit_transform(df['description'])

In [27]:
svd = TruncatedSVD(n_components=30, random_state=42)
vec_texts_svd = svd.fit_transform(vec_texts)

### KMeans

Кластеризация на 21 кластер прошла не очень успешно - распределение на кластеры совсем непохоже на распределение оценок в данных.  
Inertia score большой, а silhouette маленький - это тоже плохо.

In [28]:
kmeans = KMeans(n_clusters=21, n_init=10) 
kmeans.fit(vec_texts_svd)

KMeans(n_clusters=21, n_init=10)

In [29]:
Counter(kmeans.labels_).most_common()

[(3, 1014),
 (18, 821),
 (2, 691),
 (8, 664),
 (6, 577),
 (20, 549),
 (11, 548),
 (19, 513),
 (1, 512),
 (9, 492),
 (0, 475),
 (4, 422),
 (12, 379),
 (13, 356),
 (7, 346),
 (17, 338),
 (10, 319),
 (5, 266),
 (15, 265),
 (14, 246),
 (16, 207)]

In [30]:
inertia = kmeans.inertia_
silhouette = metrics.silhouette_score(vec_texts_svd, kmeans.labels_, metric='euclidean')
print(f'Inertia score: {inertia:.2f}')
print(f'Silhouette: {silhouette:.5f}')

Inertia score: 624.24
Silhouette: 0.04912


Кластеризация на 2 кластера получилась ненамного лучше.

In [32]:
kmeans = KMeans(n_clusters=2, n_init=10) 
kmeans.fit(vec_texts_svd)

KMeans(n_clusters=2, n_init=10)

In [33]:
Counter(kmeans.labels_).most_common()

[(1, 8102), (0, 1898)]

In [34]:
inertia = kmeans.inertia_
silhouette = metrics.silhouette_score(vec_texts_svd, kmeans.labels_, metric='euclidean')
print('Scores for 2 clusters')
print(f'Inertia score: {inertia:.2f}')
print(f'Silhouette: {silhouette:.5f}')

Scores for 2 clusters
Inertia score: 889.50
Silhouette: 0.14102


Зато можно посмотреть, что кластеризация получается намного лучше, если снижать размерность векторов текстов.  
Ниже кластеризация на tf-idf без снижения размерности.

In [35]:
kmeans = KMeans(n_clusters=2, n_init=10) 
kmeans.fit(vec_texts)
inertia = kmeans.inertia_
silhouette = metrics.silhouette_score(vec_texts, kmeans.labels_, metric='euclidean')
print(f'Inertia score: {inertia:.2f}')
print(f'Silhouette: {silhouette:.5f}')

Inertia score: 9353.37
Silhouette: 0.00380


### DBScan

DBScan не находит хорошо выделяющихся кластеров в данных, получается плохо.  
Ниже привела примеры маленьких выделенных кластеров. Человеческим глазом не очень понятно, что именно их объединяет.

In [44]:
cluster = DBSCAN(min_samples=2, eps=0.3) 
cluster.fit(vec_texts_svd)

DBSCAN(eps=0.3, min_samples=2)

In [47]:
cluster_labels = cluster.labels_
Counter(cluster_labels).most_common()

[(0, 9773),
 (-1, 205),
 (3, 4),
 (1, 2),
 (2, 2),
 (4, 2),
 (5, 2),
 (6, 2),
 (7, 2),
 (8, 2),
 (9, 2),
 (10, 2)]

In [46]:
silhouette = metrics.silhouette_score(vec_texts_svd, cluster.labels_, metric='euclidean')
print(f'Silhouette: {silhouette:.5f}')

Silhouette: 0.18359


In [56]:
inds = np.where(cluster_labels == 3)
for desc in df.iloc[inds]['description']:
  print(desc)
  print('----')

A soft, perfumed wine, with yellow fruits, green plums and an orange zest texture. It is pure and fresh, emphasizing acidity as well as fruit. There is a delicious, fresh character to this ready-to-drink wine.
----
This is a fashionably pale rosé, as light and bright in color as it is on the palate. Fruity, attractive and ready to drink, it has zesty orange peel and crisp wild strawberry flavors.
----
This pale-colored wine tastes as light as its color. It's fresh and crisp, with attractive acidity and fragrant orange and apple fruits. Soft, it's ready to drink.
----
This wine is spicy, fruity and immediately attractiv with ripe yellow fruits as well as acidity and zesty lime. Balanced and refreshing, this lively wine is ready to drink.
----


In [60]:
inds = np.where(cluster_labels == 4)
for desc in df.iloc[inds]['description']:
  print(desc)
  print('----')

This wine is juicy and fruity, with generous tannins that support its ripe black fruit and balanced acidity. It has a smoky edge and will be ready to drink from 2020.
----
Produced from organically grown grapes, this is a ripe, easy and fruity wine. It has light tannins, juicy black-currant fruit and acidity. It is still young, showing off its fruitiness and will be ready to drink soon, from early 2017.
----
